In [ ]:
# 셀프 쿼리(Self-querying)
# Self-querying retrievers 지원 db (*ElasticSearch도 있음)
# => https://python.langchain.com/v0.1/docs/integrations/retrievers/self_query/
# ElasticSearch 예제
# => https://python.langchain.com/v0.1/docs/integrations/retrievers/self_query/elasticsearch_self_query/
# 
# SelfQueryRetriever 는 자체적으로 질문을 생성하고 해결할 수 있는 기능을 갖춘 검색 도구입니다. 
# 이는 사용자가 제공한 자연어 질의를 바탕으로, query-constructing LLM chain을 사용해 구조화된 질의를 만듭니다. 
# 그 후, 이 구조화된 질의를 기본 벡터 데이터 저장소(VectorStore)에 적용하여 검색을 수행합니다.
#
# 이 과정을 통해, SelfQueryRetriever 는 단순히 사용자의 입력 질의를 저장된 문서의 내용과 의미적으로 비교하는 것을 넘어서, 
# 사용자의 질의에서 문서의 메타데이터에 대한 필터를 추출하고, 이 필터를 실행하여 관련된 문서를 찾을 수 있습니다. 
# 이를 통해, 사용자의 질의에 대한 더 정확하고 관련성 높은 결과를 제공할 수 있게 됩니다.

# 참고: SelfQueryRetriever 를 사용하려면 lark 패키지를 설치해야 합니다.
%pip install -qU lark chromadb

In [ ]:
# 루트경로에 .env 파일을 만들고, OPENAI_API_KEY='{API_KEY}' 식으로 입력한다.
# API 키를 환경변수로 관리하기 위한 .env설정 파일 로딩
import os
from dotenv import load_dotenv

load_dotenv() # API 키 정보 로드
print(f"[OPENAI_API_KEY]\n{os.environ['OPENAI_API_KEY']}\n")
print(f"[HUGGINGFACEHUB_API_TOKEN]\n{os.environ['HUGGINGFACEHUB_API_TOKEN']}")

In [2]:

import logging
# Set up the logger
logging.basicConfig(level=logging.INFO)


In [3]:
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

# 영화 내용에 대해 벡터데이터 생

docs = [
    Document(
        page_content="많은 과학자들이 공룡을 부활시키고 대혼란이 시작됩니다.e",
        metadata={"year": 1993, "평점": 7.7, "장르": "SF영화"},
    ),
    Document(
        page_content="레오 디카프리오는 꿈 속의 꿈 속의 꿈 속에서 길을 잃었습니다. ...",
        metadata={"year": 2010, "감독": "크리스토퍼 놀란", "평점": 8.2},
    ),
    Document(
        page_content="심리학자/탐정은 일련의 꿈 속에서 꿈 속의 꿈 속에서 길을 잃었고 인셉션은 그 아이디어를 재사용했습니다.",
        metadata={"year": 2006, "감독": "Satoshi Kon", "평점": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "감독": "Greta Gerwig", "평점": 8.3},
    ),
    Document(
        page_content="장난감이 살아 움직이며 신나는 시간을 보내게 됩니다.",
        metadata={"year": 1995, "장르": "에니메이션"},
    ),
    Document(
        page_content="세 남자가 구역 안으로 들어가고, 세 남자가 구역 밖으로 나간다.",
        metadata={
            "year": 1979,
            "감독": "Andrei Tarkovsky",
            "장르": "스릴러",
            "평점": 9.9,
        },
    ),
]
vectorstore = Chroma.from_documents(docs, OpenAIEmbeddings())

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [7]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI
from langchain.retrievers.self_query.chroma import ChromaTranslator

# SelfQueryRetriever 생성하기
# 이제 retriever를 인스턴스화할 수 있습니다. 이를 위해서는 문서가 지원하는 메타데이터 필드 와 문서 내용에 대한 간단한 설명을 미리 제공 해야 합니다.
# AttributeInfo 클래스를 사용하여 영화 메타데이터 필드에 대한 정보를 정의합니다.
# - 장르(genre): 문자열 타입, 영화의 장르를 나타내며 ['science fiction', 'comedy', 'drama', 'thriller', 'romance', 'action', 'animated'] 중 하나의 값을 가집니다.
# - 연도(year): 정수 타입, 영화가 개봉된 연도를 나타냅니다.
# - 감독(director): 문자열 타입, 영화 감독의 이름을 나타냅니다.
# - 평점(rating): 실수 타입, 1-10 범위의 영화 평점을 나타냅니다.

metadata_field_info = [
    AttributeInfo(
        name="장르",
        description="영화 장르를 보여준다. SF영화, 코미디, 드라마, 스릴러, 로멘스, 엑션, 에니메이션 중 1나",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="영화가 개봉한 날짜",
        type="integer",
    ),
    AttributeInfo(
        name="감독",
        description="영화 만든 감독",
        type="string",
    ),
    AttributeInfo(
        name="평점", description="1~10까지 영화평점. 높을수록 좋음.", type="float"
    ),
]

# 문서의 내용에 대한 간략한 설명
document_content_description = "영화들에 대한 간략 요약"

# LLM 정의
llm = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0)

# SelfQueryRetriever 생성
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    enable_limit=True,  # 검색 결과 제한 기능을 활성화합니다.
    search_kwargs={"k": 1},  # k 의 값을 2로 지정하여 검색 결과를 1개로 제한합니다.

    # **쿼리 변환기(ChromaTranslator를 사용하여 구조화된 질의를 Chroma 벡터 저장소에 맞게 변환합니다.)
    # => ElasticSearch 일때는 필요없음.
    # => 다른 db는 쿼리변환기 안하면 복합 필터를 사용하여 검색하는 경우 에러 발생함
    structured_query_translator=ChromaTranslator(), 
    Verbose=True,
)

In [8]:
# 8.5 이상의 평점을 받은 영화를 보고 싶다는 필터만 지정합니다.
retriever.invoke("8.5 이상 평점을 받은 영화만 보여주세요.")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[Document(page_content='심리학자/탐정은 일련의 꿈 속에서 꿈 속의 꿈 속에서 길을 잃었고 인셉션은 그 아이디어를 재사용했습니다.', metadata={'year': 2006, '감독': 'Satoshi Kon', '평점': 8.6})]

In [9]:
# Greta Gerwig가 여성에 관한 영화를 연출한 적이 있는지 질의합니다.
retriever.invoke("Has Greta Gerwig directed any movies about women")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[Document(page_content='A bunch of normal-sized women are supremely wholesome and some men pine after them', metadata={'year': 2019, '감독': 'Greta Gerwig', '평점': 8.3})]

In [ ]:
# 8.5 이상의 평점을 가진 SF 영화를 검색하는 복합 필터를 지정합니다.
# => 복합 필터: rating above 8.5, science fiction
retriever.invoke("8.5 이상의 평점을 가진 장르가 SF 영화는 어떤것이 있나요?")

In [ ]:
#이 질의 역시 복합 필터를 사용하여 검색 결과를 필터링합니다.
# => 복합 필터: 1990 ~ 2005년, 장난관 관련 영화, 에니메이션 영화 선호

# 에러가 발생하는 쿼리
retriever.invoke(
    # 1990년 이후 2005년 이전에 제작된 장난감에 관한 영화를 검색하며, 애니메이션 영화를 선호한다는 쿼리와 복합 필터를 지정합니다.
    "장남감 관련 영화이면서 장르가 에니메이션인 영화 알려주세요"
)